In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
from sklearn.linear_model import LinearRegression

from scipy import stats

# Initialize seaborn (for plotting)
sns.set()

In [17]:
df_Centers = pd.read_csv("centers.csv", index_col=0)
df_Demands = pd.read_csv("demands.csv")
df_Distances = pd.read_csv("distances.csv")

In [18]:
Regions = df_Demands['Region'].unique()
new_years = np.array([2023,2024,2025])
new_X = pd.DataFrame({'X': list(new_years),'X^2':list(new_years**2)})
predictions = []
Region = []
Year = []
for region in Regions:
    df = df_Demands[df_Demands['Region'] == region]
    y = df['Demand'].reset_index(drop=True)
    X = df['Year']
    data = {'X': list(X),'X^2': list(X**2)}
    X = pd.DataFrame(data)
    model = LinearRegression()
    model.fit(X, y)
    prediction = model.predict(new_X)
    predictions = np.append(predictions, prediction)
    Region = np.append(Region, np.array([region]*3))
    Year = np.append(Year, np.array(new_years))

df_prediction = pd.DataFrame({'Year': Year, 'Region': Region, 'Pred_Demand': predictions})
df_prediction['Year'] = df_prediction['Year'].astype(int)
df_prediction

,Year,Region,Pred_Demand
0,2023,Downtown Toronto,406126.666667
1,2024,Downtown Toronto,426154.769697
2,2025,Downtown Toronto,446985.690909
3,2023,East York,281076.383333
4,2024,East York,294831.374242
5,2025,East York,309115.963636
6,2023,Etobicoke,467217.900000
7,2024,Etobicoke,490055.457576
8,2025,Etobicoke,513748.454545
9,2023,Mississauga East,414563.233333


In [19]:
import math
def round_up(x):
    return math.ceil(x)

In [20]:
df_prediction_round = df_prediction.copy()

In [21]:
df_prediction_round["Pred_Demand"] = df_prediction["Pred_Demand"].apply(round_up)
df_prediction_round.sort_values("Year")

,Year,Region,Pred_Demand
0,2023,Downtown Toronto,406127
18,2023,Scarborough East,455085
3,2023,East York,281077
15,2023,North York,518490
6,2023,Etobicoke,467218
12,2023,North Toronto,425697
9,2023,Mississauga East,414564
16,2024,North York,543579
13,2024,North Toronto,446171
19,2024,Scarborough East,477079


In [22]:
# Centers
centers = list(df_Centers.index)
num_centers = len(centers)
print(centers)
print("Total number of centers: " + str(num_centers))

# Regions
regions = list(df_Demands['Region'].unique())
num_regions = len(Regions)
print(regions)
print("Total number of regions: " + str(num_regions))

# Periods
years = list(df_prediction['Year'].unique())
num_years = len(years)
print(years)
print("Total number of years: " + str(num_years))

['Center A', 'Center B', 'Center C', 'Center D', 'Center E', 'Center F']
Total number of centers: 6
['Downtown Toronto', 'East York', 'Etobicoke', 'Mississauga East', 'North Toronto', 'North York', 'Scarborough East']
Total number of regions: 7
[2023, 2024, 2025]
Total number of years: 3


### Constants

In [23]:
# Hourly pay for a worker
hour_pay = 37.85

# pay growth rate
growth_rate = 0.025

# travel expenses for a worker cost per km per hour
cost_per_km = 0.25

# government reimbursement
gov_reimb = 42

# max hiring:
max_hiring = 300

# hiring cost per MP
cost_hiring = 15000

# Must open centers:
opened_centers = df_Centers.index[0:4]

# Optional opening centers:
not_open_centers = df_Centers.index[4:]

# Max working hour per day per MP
max_hour = 6

# 3 Max working day per year per MP
max_day = 250

# total number of workers from year 1:
total_worker_in_year1 = sum(df_Centers["Num MPs"])

### Parameters

In [24]:
# Demand in region i at time t (dictionary) Dit
region_demand = {}
for j in regions:
    for t in years:
        region_demand[(j,t)] = float(df_prediction_round[(df_prediction_round['Region'] == j) & 
                                                   (df_prediction_round['Year'] == t)]['Pred_Demand'])

# Operating cost for each center fi
operating_cost = {}
for i in centers:
    operating_cost[i] = float(df_Centers.loc[i]["Operating Costs"])

# Opening cost for each center Oi
opening_cost = {}
for i in centers:
    opening_cost[i] = float(df_Centers.loc[i]["Opening Costs"])

# Center capacities (dictionary) Ki
center_capacity = {}
for i in centers:
    center_capacity[i] = df_Centers.loc[i]["Maximum capacity"]

# Cost for workers from center i to work in region j at time t   Cijt
cost_region_centers = {}
for t in years: 
    for i in centers:
        for j in regions:
            cost_region_centers[(i,j,t)] = hour_pay*((1 + growth_rate)**(t-years[0])) - gov_reimb + \
            cost_per_km * (df_Distances[(df_Distances["Center"] == i) & (df_Distances["Region"] == j)]["Distance"])

### Variables

In [25]:
# Variables: if a center is opened/allocated
yvar = pulp.LpVariable.dict("y", (centers, years), cat=pulp.LpBinary)

# Variables: amount of demand from each region allocated to center
xvar = pulp.LpVariable.dict("x", (centers, regions, years), lowBound=0.0, cat=pulp.LpContinuous)

# Variables: number of workers allocated to center i
wvar = pulp.LpVariable.dict("w", (centers, years), lowBound= 0, cat = pulp.LpInteger)

# Variables: number of workers to hire for center i
hvar = pulp.LpVariable.dict("h", (centers, years), lowBound= 0, upBound= max_hiring, cat= pulp.LpInteger)

### Model initialization

In [26]:
# Initialize model and objective sense
final_model = pulp.LpProblem(name="Final_Model", sense=pulp.LpMinimize)

### Constraints

In [27]:
# Constraint: demand must be satisfied for every region and all 3 years
for t in years:
    for j in regions:
        final_model += pulp.lpSum( [xvar[(i,j,t)] for i in centers] ) >= 0.443 * region_demand[(j,t)]

for t in years:
    for j in regions:
        final_model += pulp.lpSum( [xvar[(i,j,t)] for i in centers] ) <= region_demand[(j,t)]

# Constraint: center max capacities must be observed in all scenarios
for t in years:
    for i in centers:        
        final_model += pulp.lpSum( [xvar[(i,j,t)] for j in regions] ) <= center_capacity[i] * yvar[(i,t)]

# Constraint: centers A ~ D must be open
#for t in years:
#    final_model += pulp.lpSum( [yvar[(i, t)] for i in opened_centers]) == 4

# Constraint: opened centers must stay open
#for t in years[1:]:
#     for i in centers:
#        final_model += yvar[(i, t)] == yvar[(i, t-1)] 

# Constraint: total number of workers = 560 for first year
final_model += pulp.lpSum( [wvar[(i,years[0])] for i in centers]) == total_worker_in_year1

# Constraint: total demand served must be smaller than total working hour 
for t in years:
    for i in centers: 
        final_model += pulp.lpSum([xvar[(i,j,t)] for j in regions]) <= max_hour * max_day * (wvar[(i,t)] + hvar[(i,t)])

# Constriant: total number of workers at t for center i should be the total number of workers at t-1 + hired at t-1
for t in years[1:]:
    for i in centers:
        final_model += wvar[(i,t-1)] + hvar[(i,t-1)] == wvar[(i,t)]

### Objective Function

In [28]:
# Objective Function

# Total Operating over the years
obj = pulp.lpSum([ operating_cost[i] * yvar[(i,t)] for t in years for i in centers])

# Total Opening Costs for the first year
obj += pulp.lpSum([ opening_cost[i] * yvar[(i,years[0])] for i in centers])

# Total cost for service
obj += pulp.lpSum([ cost_region_centers[(i,j,t)] * xvar[(i,j,t)] for i in centers for j in regions for t in years] )

# Total hiring costs
obj += pulp.lpSum( [hvar[(i,t)] for i in centers for t in years]) * cost_hiring

final_model += obj

In [29]:
from pulp import PULP_CBC_CMD

In [30]:
final_model.writeLP("Final_Model.lp")
# 
final_model.solve(PULP_CBC_CMD(msg=1, maxSeconds= 60*3))
print("Status:", pulp.LpStatus[final_model.status])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/lc/b3fnl0s565b49nnwtc7k9r_40000gn/T/1bd704b4d62246e0b497a2aa4e6913dd-pulp.mps sec 180 timeMode elapsed branch printingOptions all solution /var/folders/lc/b3fnl0s565b49nnwtc7k9r_40000gn/T/1bd704b4d62246e0b497a2aa4e6913dd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 96 COLUMNS
At line 967 RHS
At line 1059 BOUNDS
At line 1114 ENDATA
Problem MODEL has 91 rows, 180 columns and 600 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 180
Option for timeMode changed from cpu to elapsed
Continuous objective value is -2.6974e+06 - 0.00 seconds
Cgl0004I processed model has 64 rows, 174 columns (48 integer (18 of which binary)) and 468 elements
Cbc0038I Initial state - 21 integers unsatisfied sum - 4.16492
Cbc0038I Pass   1

In [31]:
# Total cost
totalCost = pulp.value(final_model.objective)
print("Total cost: " + str(totalCost))

Total cost: -1343.6845634328201


In [32]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [33]:
total_operating = sum([operating_cost[i] * -yvar[(i,t)].varValue for t in years for i in centers])
total_opening = sum([ opening_cost[i] * - yvar[(i,years[0])].varValue for i in centers])
total_service_cost = sum([-((cost_region_centers[(i,j,t)].values)[0] + 42) * xvar[(i,j,t)].varValue for i in centers for j in regions for t in years])
total_hiring_cost = sum([hvar[(i,t)].varValue for i in centers for t in years]) * - cost_hiring
total_service_revenue = sum( [xvar[(i,j,t)].varValue * 42 for i in centers for j in regions for t in years])
result_summary = pd.DataFrame({"total_operating_cost": total_operating, "total_opening" :total_opening,"total_service_cost": total_service_cost,
                               "total_hiring_cost": total_hiring_cost,
                               "total_service_revenue" : total_service_revenue, "total_Profit_/_Cost":-totalCost},index=[0])
result_summary = result_summary.T.rename(columns = {0:"In dollar $"})
result_summary = result_summary.round(2)
result_summary

,In dollar $
total_operating_cost,-2550000.00
total_opening,-350000.00
total_service_cost,-173408656.74
total_hiring_cost,-6075000.00
total_service_revenue,182385000.42
total_Profit_/_Cost,1343.68


In [34]:
cost_no_hiring = total_service_revenue + total_service_cost + total_operating + total_opening 
cost_no_hiring

6076343.684563428

In [35]:
# Print solution (you can add to the dataframe if needed)
for t in years:
    print("Year = " + str(t))
    for i in centers:
        if yvar[(i,t)].varValue >= 1.0:
            print("\tCenter " + str(i) + " is operating")        
            print("\t\tAllocated Worker = " +str(wvar[(i,t)].varValue))
            print("\t\tNew hires = " +str(hvar[(i,t)].varValue))              
            for j in regions:
                if xvar[(i,j,t)].varValue > 0.0:
                    print("\t\t\tServes region " + str(j) + " - with capacity " + str(xvar[(i,j,t)].varValue))
        else:
            print("\tCenter " + str(i) + " is not operating") 

Year = 2023
	Center Center A is operating
		Allocated Worker = 5.0
		New hires = 255.0
			Serves region East York - with capacity 124517.11
			Serves region North York - with capacity 265482.89
	Center Center B is operating
		Allocated Worker = 0.0
		New hires = 148.0
			Serves region Scarborough East - with capacity 222000.0
	Center Center C is not operating
	Center Center D is not operating
	Center Center E is operating
		Allocated Worker = 420.0
		New hires = 2.0
			Serves region Downtown Toronto - with capacity 179914.26
			Serves region Etobicoke - with capacity 206977.57
			Serves region North Toronto - with capacity 246108.17
	Center Center F is operating
		Allocated Worker = 135.0
		New hires = 0.0
			Serves region Mississauga East - with capacity 202500.0
Year = 2024
	Center Center A is operating
		Allocated Worker = 260.0
		New hires = 0.0
			Serves region East York - with capacity 130610.58
			Serves region North York - with capacity 259389.42
	Center Center B is operating
	

In [36]:
# Print solution (you can add to the dataframe if needed)
Year = []
num_HPs = []
num_new_hire = []
Center = []
Open_Y = []

East_York = []
Etobicoke = []
Mississauga_East = []
North_Toronto= []
North_York = []
Scarborough_East = []
Downtown_Toronto = []


for t in years:
    for i in centers:
        Year = np.append(Year, t)
        num_HPs = np.append(num_HPs, wvar[(i,t)].varValue)     
        num_new_hire = np.append(num_new_hire, hvar[(i,t)].varValue)    
        Center =  np.append(Center, i)
        Open_Y = np.append(Open_Y, yvar[(i,t)].varValue)
        for j in regions:
            if j == 'East York':
                East_York = np.append(East_York, xvar[(i,j,t)].varValue)
            elif j == 'Etobicoke':
                Etobicoke = np.append(Etobicoke, xvar[(i,j,t)].varValue)
            elif j == 'Mississauga East':
                Mississauga_East = np.append(Mississauga_East, xvar[(i,j,t)].varValue)
            elif j == 'North Toronto':
                North_Toronto = np.append(North_Toronto, xvar[(i,j,t)].varValue)
            elif j == 'North York':
                North_York = np.append(North_York, xvar[(i,j,t)].varValue)
            elif j == 'Scarborough East':
                Scarborough_East = np.append(Scarborough_East, xvar[(i,j,t)].varValue)
            elif j == 'Downtown Toronto':
                Downtown_Toronto = np.append(Downtown_Toronto, xvar[(i,j,t)].varValue)
        

In [37]:
result_table = pd.DataFrame({"Center": Center, "Open_Y": Open_Y, "Year": Year, "num_HPs": num_HPs, "num_new_hire": num_new_hire, "East_York": East_York,
                              "Etobicoke": Etobicoke, "Mississauga_East":Mississauga_East, "North_Toronto": North_Toronto,
                              "North_York":North_York, "Scarborough_East":Scarborough_East,"Downtown_Toronto":Downtown_Toronto})
result_table["Year"] = result_table["Year"].astype(int)
result_table["num_HPs"] = result_table["num_HPs"].astype(int)
result_table["num_new_hire"] = result_table["num_new_hire"].astype(int)
result_table = result_table.round(decimals = 2)
result_table

,Center,Open_Y,Year,num_HPs,num_new_hire,East_York,Etobicoke,Mississauga_East,North_Toronto,North_York,Scarborough_East,Downtown_Toronto
0,Center A,1.00,2023,5,255,124517.11,0.00,0.00,0.00,265482.89,0.00,0.00
1,Center B,1.00,2023,0,148,0.00,0.00,0.00,0.00,0.00,222000.00,0.00
2,Center C,0.00,2023,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Center D,0.00,2023,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Center E,1.00,2023,420,2,0.00,206977.57,0.00,246108.17,0.00,0.00,179914.26
5,Center F,1.00,2023,135,0,0.00,0.00,202500.00,0.00,0.00,0.00,0.00
6,Center A,1.00,2024,260,0,130610.58,0.00,0.00,0.00,259389.42,0.00,0.00
7,Center B,1.00,2024,148,0,0.00,0.00,0.00,0.00,0.00,222000.00,0.00
8,Center C,0.00,2024,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Center D,0.00,2024,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [537]:
total_satisifed_demand = (result_table[["East_York","Etobicoke","Mississauga_East","North_Toronto","North_York","Scarborough_East","Downtown_Toronto"]].sum(axis=0)).sum()
total_demand = sum(df_prediction_round["Pred_Demand"])
1-(total_demand - total_satisifed_demand)/total_demand

0.26975090430779347

9341952

0.0